## Логика игры

Приветствие и правила игры

Показ игровой доски ИИ (все в нулях)

Создание рандомных кораблей на доске ИИ

Запрос создания кораблей Игрока (показ при каждом размещении кораблей)

### Loop:
Ход игрока

Проверка правильности хода

Размещение на доске ИИ и отображение его доски

Проверка на ПОБЕДУ

Ход ИИ и отображение доски Игрока

Проверка на ПОБЕДУ


# ...Error Classes

In [1]:
class BoardOutError(Exception):
    def __init__(self,value):
        self.value = value
    
    def __str__(self):
        return f"{self.value} лежит за пределами игрового поля. Введите двузначное число, каждая цифра которого от 1 до 6, включительно."

class NonEmptyError(Exception):
    def __init__(self,value):
        self.value = value
    
    def __str__(self):
        return f"Выбраная ячейка - {self.value} - непустая. Стрелять можно только в пустые ячейки."

class ShipLocationError(Exception):
    def __init__(self,length, front, direction):
        self.length = length
        self.front = front
        self.direction = direction
    
    def __str__(self):
        return f"Указанные параметры - длина ({self.length}), положение носа корабля ({self.front}) и направление ({self.direction}) не позволяют корректно разместить корабль на игровом поле."
    

# ...Board, Ship, Dot Classes

In [2]:
class Dot:
    def __init__(self, row, col):
        self.row = row
        self.col = col
        
    def __eq__(self, other):
        return self.row == other.row and self.col == other.col
    
    @property
    def show(self):
        return(self.row, self.col)

In [14]:
grid = [['o' for i in range(6)] for i in range(6)]

def print_grid(L):
    print(*[" ", *[i for i in range(1,7)]], sep=' | ')
    for i in range(6):
        print(i+1, *L[i], sep=' | ')
    return

In [2]:
class Ship:
    def __init__(self, length, front, direction, life):
        self.length = length
        self.front = front
        self.direction = direction
        self.life = life
    
    def dots(self):
        ship_cells = []
        for i in range(self.length):
            cell = (self.front[0] + self.direction[1]*(i), self.front[1] + self.direction[0]*(i))
            ship_cells.append(cell)
        return ship_cells


In [3]:
ship_1 = Ship(3,(3,3),(1,0),3)
print(ship_1.dots())

[(3, 3), (3, 4), (3, 5)]


In [51]:
class Board:
    POSSIBLE_CELLS = []
    for i in range(6):
        for j in range(6):
            POSSIBLE_CELLS.append((i,j))
    print(POSSIBLE_CELLS)
    
    def __init__(self, cells_conditions, ships, hid, alive_ships):
        self.cells_conditions = cells_conditions    # list
        self.ships = ships                          # list with all ships
        self.hid = hid                              # bool
        self.alive_ships = alive_ships              # int
    
   
    def add_ship(self, ship_dots):      #ставит корабль на доску (если ставить не получается, выбрасываем исключения)
        for i in len(ship_dots):
            if ship_dots[i] not in ships and ship_dots[i] not in self.contour(self.ships):
                ships
            
    
#     def contour(self):      #обводит корабль по контуру
#         pass
        
    def show_board(self):      #выводит доску в консоль в зависимости от параметра hid.
        if not hid:
            return self.ships
    
    def out(self, dot):      #для точки (объекта класса Dot) возвращает True, если точка выходит за пределы поля
        if dot in self.POSSIBLE_CELLS:
            return False
        else:
            return True
    
    def shot(self):         #делает выстрел по доске (если есть попытка выстрелить за \
                            #пределы и в использованную точку, нужно выбрасывать исключения
        while True:         #Loop для получения правильного ввода от Игрока
            try:
                player_shot = int(input("В какую ячейку будем стрелять?"))
                row = player_shot // 10
                col = player_shot % 10

                if self.out((row-1, col-1)):
                    err = BoardOutError(player_shot)
                    print(err)
                else:
                    player_shots_left = [Dot(1,1),Dot(2,2)]
                    shot = Dot(row,col)
                    
                    if shot not in player_shots_left:
                        err = NonEmptyError(player_shot)
                        print(err)
                    else:
                        if self.ships[row-1][col-1] == 'o':
                            self.cells_conditions[row-1][col-1] = 'T'
                            player_shots_left.remove(Dot(row,col))
                            print(self.cells_conditions)
                        else:
                            self.cells_conditions[row-1][col-1] = 'X'
                            player_shots_left.remove(Dot(row,col))
                            print(self.cells_conditions)
                        print('Ход компьютера:')
                        break

            except Exception:
                print('Необходимо ввести двузначное число!')

player = 'Player'

cells_conditions = [['o','o','o'],['o','o','o'],['o','o','o']]
ships = [['o','o','S'],['o','S','o'],['S','o','o']]
hid = True if player == 'AI' else False
alive_ships = 1

AI_board = Board(cells_conditions, ships, hid, alive_ships)
AI_board.shot()

[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5)]
В какую ячейку будем стрелять?12
Выбраная ячейка - 12 - непустая. Стрелять можно только в пустые ячейки.
В какую ячейку будем стрелять?11
[['T', 'o', 'o'], ['o', 'o', 'o'], ['o', 'o', 'o']]
Ход компьютера:


In [ ]:
ships = [[(1,1)],
         [

In [ ]:
ship_1 = Ship(3,(2,2),(0,1),3)
print(ship_1)
ship_1_cells = ship_1.dots()
print(ship_1_cells)

for i in range(len(ship_1_cells)):
    row = ship_1_cells[i][0]
    col = ship_1_cells[i][1]
    print(row, col)
    
    grid[row-1][col-1] = "X"

print_grid(grid)

In [52]:
AI_board.player_shots_left

AttributeError: 'Board' object has no attribute 'player_shots_left'

In [ ]:
while True:      #Loop для получения правильного ввода от Игрока
    try:
        user_shot = int(input("В какую ячейку будем стрелять?"))
        row = user_shot // 10
        col = user_shot % 10

        if row not in range(1,7) or col not in range(1,7):
            err = BoardOutError(user_shot)
            print(err)

        else:
            shot = Dot(row,col)
            if shot in user_shots:
                err = NonEmptyError(user_shot)
                print(err)
            else:
                grid[row-1][col-1] = 'X'
                user_shots.append(shot)
                print_grid(grid)
                print('Ход компьютера:')
                break

    except Exception:
        print('Необходимо ввести двузначное число!')

In [ ]:
for i in range(len(user_shots)):
    print(i, user_shots[i].rtrn())
user_shots[2] == user_shots[2]


In [ ]:
# from random import randint

# while True:
#     row, col = randint(1,6),randint(1,6)
#     print(row,col)
#     shot = Dot(row, col)
#     if shot not in PC_shots:
#         grid[row-1][col-1] = 'X'
#         PC_shots.append(shot)
#         print_grid(grid)
#         break

unused_shots = []
for i in range(6):
    for j in range(6):
        unused_shots.append((i,j))
                               
                               
print(unused_shots[randint(1,len(unused_shots))])


In [ ]:
s = u'\u220E'     #код квадрата в системе unicode 


In [46]:
[[j for j in range(5)] for i in range(5)]

[[0, 1, 2, 3, 4],
 [0, 1, 2, 3, 4],
 [0, 1, 2, 3, 4],
 [0, 1, 2, 3, 4],
 [0, 1, 2, 3, 4]]

In [ ]:
grid = [['o' for i in range(6)] for i in range(6)]

def print_grid(L):
    print(*[" ", *[i for i in range(1,7)]], sep=' | ')
    for i in range(6):
        print(i+1, *L[i], sep=' | ')
    return

# print_grid(grid)

us_in = 55
row = us_in // 10
col = us_in % 10
grid[row-1][col-1] = "X"
print_grid(grid)

grid_user = grid.copy()
grid[3][3] = "X"
print_grid(grid_user)
print(id(grid_user))
print(id(grid))

In [ ]:
print(user_shots)

In [ ]:
user_shots[0] == user_shots[1]

In [ ]:
# Python program showing the use of
# @property
  
class Geeks:
     def __init__(self):
          self._age = 0
       
     # using property decorator
     # a getter function
     @property
     def age(self):
         print("getter method called")
         return self._age
       
     # a setter function
     @age.setter
     def age(self, a):
         if(a < 18):
            err = ValueError("Sorry you age is below eligibility criteria")
            print(err)
         print("setter method called")
         self._age = a
  
mark = Geeks()
print(mark.age)
mark.age = int(input())
  
print(mark.age)

# ВНЕШНЯЯ ЛОГИКА

In [ ]:
class Player:
    def __init__(self, )
    
    @property
    def my_board(self):
        self.xxx = Board(grid_ships)
    
    @property
    def opponent_board(self):
        self.xxx = Board(grid_ships)
        
    def ask(self):
        pass
    
    def move(self):
        while True:
            self.Player.ask()
            
            